In [1]:
# ====== Exploración de datos en NCBI con Biopython =========
# Carga de bibliotecas
from Bio import Entrez # Permite conectarse con las bases de datos de NCBI ,explorar y descargar información sobre proyectos, muestras y datos de secuenciación.
import pandas as pd
from io import StringIO
import subprocess
import os

Entrez.email = "darapc@chaac.lcg.unam.mx"

In [2]:
# Obtener el UID de un BioProject
# Resumen del BioProject
project_acc = "PRJNA482464"
handle = Entrez.esearch(db="bioproject", term=project_acc)
search_results = Entrez.read(handle)
print(search_results)
handle.close()

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['482464'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'PRJNA482464[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA482464[All Fields]'}


In [3]:
# Extraer solo el UID
project_uid = search_results["IdList"][0]
print("UID del BioProject:", project_uid)

UID del BioProject: 482464


In [4]:
# Resumen del BioProject
handle = Entrez.esummary(db="bioproject", id=project_uid)
proj_record = Entrez.read(handle)
print(proj_record)
handle.close()


{'DocumentSummarySet': DictElement({'DocumentSummary': [DictElement({'TaxId': '3885', 'Project_Id': '482464', 'Project_Acc': 'PRJNA482464', 'Project_Type': 'Primary submission', 'Project_Data_Type': 'Raw sequence reads', 'Sort_By_ProjectType': '626794', 'Sort_By_DataType': '616470', 'Sort_By_Organism': '504137', 'Project_Subtype': '', 'Project_Target_Scope': 'Multiisolate', 'Project_Target_Material': 'Genome', 'Project_Target_Capture': 'Whole', 'Project_MethodType': 'Sequencing', 'Project_Method': '', 'Project_Objectives_List': [{'Project_ObjectivesType': 'Raw Sequence Reads', 'Project_Objectives': ''}], 'Registration_Date': '2018/07/23 00:00', 'Project_Name': 'Phaseolus vulgaris strain:Rhizobium tropici and Rhizophagus irregularis', 'Project_Title': 'Phaseolus vulgaris Transcriptome after rhizobial and mycorrhizal fungi inoculation', 'Project_Description': 'Phaseolus vulgaris transcriptome reprogramming by modulating the expression of the NADPH oxidase gene RbohB after rhizobial and m

In [5]:
# Seleccionando la información relacionada al proyecto
proj = proj_record["DocumentSummarySet"]["DocumentSummary"][0]

# Mostrando todos los campos del diccionario
print(proj.keys())

# Accediendo a algunos campos
print("\n=== Información del BioProject ===")
print("Acceso:", proj["Project_Acc"])
print("Título:", proj["Project_Title"])
print("Organismo:", proj["Organism_Name"])
print("Descripción:", proj.get("Project_Description"))

dict_keys(['TaxId', 'Project_Id', 'Project_Acc', 'Project_Type', 'Project_Data_Type', 'Sort_By_ProjectType', 'Sort_By_DataType', 'Sort_By_Organism', 'Project_Subtype', 'Project_Target_Scope', 'Project_Target_Material', 'Project_Target_Capture', 'Project_MethodType', 'Project_Method', 'Project_Objectives_List', 'Registration_Date', 'Project_Name', 'Project_Title', 'Project_Description', 'Keyword', 'Relevance_Agricultural', 'Relevance_Medical', 'Relevance_Industrial', 'Relevance_Environmental', 'Relevance_Evolution', 'Relevance_Model', 'Relevance_Other', 'Organism_Name', 'Organism_Strain', 'Organism_Label', 'Sequencing_Status', 'Submitter_Organization', 'Submitter_Organization_List', 'Supergroup'])

=== Información del BioProject ===
Acceso: PRJNA482464
Título: Phaseolus vulgaris Transcriptome after rhizobial and mycorrhizal fungi inoculation
Organismo: Phaseolus vulgaris
Descripción: Phaseolus vulgaris transcriptome reprogramming by modulating the expression of the NADPH oxidase gene Rb

In [6]:
# ===== Relacionando BioProject con GEO (GSE/GSM) ====
# Revisar enlaces desde BioProject hacia GEO (GDS), con el uid del bioproject
handle = Entrez.elink(dbfrom="bioproject", db="gds", id=project_uid)
linksBioProj = Entrez.read(handle)
handle.close()

#print(linksBioProj)

if linksBioProj[0]["LinkSetDb"]:
    geo_ids = [link["Id"] for link in linksBioProj[0]["LinkSetDb"][0]["Link"]]
    print("\nEl proyecto", project_acc, " está asociado a la base de datos de GEO")
    print("UIDs GEO asociados:", geo_ids[:5], "... total:", len(geo_ids))
else:
    print("\nEl proyecto", project_acc, " NO tiene enlaces directos a GEO")


El proyecto PRJNA482464  NO tiene enlaces directos a GEO


In [7]:
# ====== Relacionando BioProject con SRA (SRR) ====
# Revisar enlaces desde Bioproject hacia SRA (sra), con el uid del bioproject
handle = Entrez.elink(dbfrom="bioproject", db="sra", id=project_uid)  # ← cambio clave: es elink
linksBioProj_sra = Entrez.read(handle)
handle.close()

# print(linksBioProj_sra)
if linksBioProj_sra and linksBioProj_sra[0].get("LinkSetDb") and linksBioProj_sra[0]["LinkSetDb"]:
    sra_ids = [link["Id"] for link in linksBioProj_sra[0]["LinkSetDb"][0]["Link"]]
    print("\nEl proyecto", project_acc, "está asociado a la base de datos de SRA")
    print("UIDs SRA asociados:", sra_ids[:len(sra_ids)], "... total:", len(sra_ids))
else:
    print("\nEl proyecto", project_acc, "NO tiene enlaces directos a SRA")


El proyecto PRJNA482464 está asociado a la base de datos de SRA
UIDs SRA asociados: ['6163728', '6163727', '6163726', '6163347', '6163346', '6163345', '6163343', '6163342', '6163341', '6163339', '6163338', '6163337', '6163333', '6163332', '6163331', '6161033', '6161032', '6161031'] ... total: 18


In [8]:
if sra_ids:
    # Convertir la lista de UIDs en una cadena separada por comas, como espera Entrez
    sra_id_list = ",".join(sra_ids)

    handle_fetch = Entrez.efetch(db="sra", id=sra_id_list, rettype="runinfo", retmode="text")
    run_info_text = handle_fetch.read().decode('utf-8') # Decode the bytes object to a string
    handle_fetch.close()

    # Ahora procesamos la información para extraer los SRR
    # El archivo runinfo tiene el formato CSV/TSV, donde la primera columna es el SRR
    srr_list = []

    # saltamos la primera línea (encabezados)
    for line in run_info_text.splitlines()[1:]:
        # La primera columna (antes de la primera coma) es el SRR
        srr_accession = line.split(",")[0]
        srr_list.append(srr_accession)

    print("\n==============================================")
    print("Identificadores SRR obtenidos:")
    print(srr_list[:len(sra_ids)], "... total:", len(srr_list))
    print("El primer SRR es:", srr_list[0]) # Ej: SRR8904481
    print("==============================================")


Identificadores SRR obtenidos:
['SRR7693916', 'SRR7693917', 'SRR7693915', 'SRR7696192', 'SRR7696193', 'SRR7696194', 'SRR7696202', 'SRR7696201', 'SRR7696200', 'SRR7696205', 'SRR7696206', 'SRR7696204', 'SRR7696210', 'SRR7696208', 'SRR7696209', 'SRR7696591', 'SRR7696590', 'SRR7696589'] ... total: 18
El primer SRR es: SRR7693916


In [9]:
# 1. Separar el texto en líneas para acceder a los encabezados
lines = run_info_text.splitlines()

if len(lines) > 1:
    # Obtener encabezados (primera línea)
    headers = lines[0].split(',') 
    
    # 2. Buscar el índice de la columna 'LibraryLayout'
    try:
        layout_index = headers.index("LibraryLayout")
    except ValueError:
        layout_index = -1
        print("\nAdvertencia: No se encontró la columna 'LibraryLayout' en los metadatos.")

    # 3. Obtener el layout del primer registro de datos (segunda línea)
    library_layout = "Desconocido"

    if layout_index != -1:
        # La segunda línea (índice 1) contiene el primer registro de datos
        first_data_fields = lines[1].split(",")
        library_layout = first_data_fields[layout_index]

        print("\n--- Verificación del Layout de Secuenciación ---")
        print(f"Library Layout detectado: **{library_layout}**")

        if "PAIRED" in library_layout.upper():
            print("--> Conclusión: ¡Los datos son de LECTURAS APAREADAS (PAIRED-END)!")
            print("    El script de conversión en Bash DEBE buscar y comprimir archivos **_1.fastq** y **_2.fastq**.")
        elif "SINGLE" in library_layout.upper():
            print("--> Conclusión: Los datos son de LECTURA SIMPLE (SINGLE-END).")
            print("    El script de Bash solo encontrará y comprimirá el archivo **_1.fastq**.")
        else:
            print("--> Advertencia: El layout no es estándar. Usaremos el script más robusto (que verifica la existencia de _2).")
        print("-------------------------------------------------")
    


--- Verificación del Layout de Secuenciación ---
Library Layout detectado: **PAIRED**
--> Conclusión: ¡Los datos son de LECTURAS APAREADAS (PAIRED-END)!
    El script de conversión en Bash DEBE buscar y comprimir archivos **_1.fastq** y **_2.fastq**.
-------------------------------------------------


In [10]:
# Define el nombre que tendrá el archivo de texto
output_filename = "srr_ids.txt"

# Asegúrate de que tu srr_list ya se ha generado correctamente
# Ejemplo (solo para referencia): srr_list = ["SRR1234567", "SRR7654321", "SRR8888888"]

# Abrimos el archivo en modo escritura ('w')
with open(output_filename, 'w') as f:
    # Escribimos cada código SRR en una nueva línea
    for srr in srr_list:
        f.write(srr + '\n')

# Confirmación de que el archivo se ha creado
print(f"Archivo generado: '{output_filename}' con {len(srr_list)} códigos SRR.")
print("Ahora puedes usar este archivo para descargar en la terminal.")

Archivo generado: 'srr_ids.txt' con 18 códigos SRR.
Ahora puedes usar este archivo para descargar en la terminal.



PARADOS EN SRC  # ---- CODIGO PARA BASH

OUTPUT_DIR="../data/raw/"
mkdir -p "$OUTPUT_DIR"

while read srr; do
    echo "Descargando: $srr..."
    prefetch --max-size u --output-directory "$OUTPUT_DIR" "$srr"
done < srr_ids.txt

para  verificar que todo se instalo : find ../data/raw -name "*.sra" | wc -l

In [15]:
# ==== Descarga del genoma de referencia =====
# Asignando el organismo
organism = "Phaseolus vulgaris"

# Consultando la base de datos assembly
handle = Entrez.esearch(db="assembly", term=organism, retmax=150)
search_results = Entrez.read(handle)
handle.close()

# Imprimiendo los IDs
assembly_uids = search_results["IdList"]
print("UIDs de ensamblaje encontrados:", assembly_uids)

UIDs de ensamblaje encontrados: ['29544511', '29284811', '28969501', '23088951', '23088921', '23088791', '23088741', '22439661', '16290251', '9059211', '9059191', '8660681', '628671', '82691']


In [16]:
# Para cada UID obtener la información
assembly_data = []

for uid in assembly_uids:
    handle = Entrez.esummary(db="assembly", id=uid)
    summary = Entrez.read(handle)
    handle.close()
    
    docsum = summary['DocumentSummarySet']['DocumentSummary'][0]
    
    # Guardamos todo el diccionario
    assembly_data.append(docsum)

In [17]:
# Crear DataFrame con toda la información
df_assembly = pd.DataFrame(assembly_data)

# Mostrar todas las columnas disponibles
display("Columnas disponibles:", df_assembly.columns.tolist())

'Columnas disponibles:'

['RsUid',
 'GbUid',
 'AssemblyAccession',
 'LastMajorReleaseAccession',
 'LatestAccession',
 'ChainId',
 'AssemblyName',
 'UCSCName',
 'EnsemblName',
 'Taxid',
 'Organism',
 'SpeciesTaxid',
 'SpeciesName',
 'AssemblyType',
 'AssemblyStatus',
 'AssemblyStatusSort',
 'WGS',
 'GB_BioProjects',
 'GB_Projects',
 'RS_BioProjects',
 'RS_Projects',
 'BioSampleAccn',
 'BioSampleId',
 'Biosource',
 'Coverage',
 'PartialGenomeRepresentation',
 'Primary',
 'AssemblyDescription',
 'ReleaseLevel',
 'ReleaseType',
 'AsmReleaseDate_GenBank',
 'AsmReleaseDate_RefSeq',
 'SeqReleaseDate',
 'AsmUpdateDate',
 'SubmissionDate',
 'LastUpdateDate',
 'SubmitterOrganization',
 'RefSeq_category',
 'AnomalousList',
 'ExclFromRefSeq',
 'PropertyList',
 'FromType',
 'Synonym',
 'ContigN50',
 'ScaffoldN50',
 'AnnotRptUrl',
 'FtpPath_GenBank',
 'FtpPath_RefSeq',
 'FtpPath_Assembly_rpt',
 'FtpPath_Stats_rpt',
 'FtpPath_Regions_rpt',
 'Busco',
 'SortOrder',
 'Meta']

In [18]:
# Seleccionar solo algunas columnas de interés
df_selected = df_assembly[[
    "Organism",
    "AssemblyName",
    "AssemblyStatus",
    "AssemblyAccession",
    "RefSeq_category",
    "FtpPath_GenBank"
]]

print("Mostrando los primeros 10 renglones del dataframe")
display(df_selected[0:10])


# Filtrar filas que coincidan exactamente con el accession
df_filtered = df_selected[df_selected["RefSeq_category"] != "na"]

# Mostrar resultado
print("Mostrando los genomas que tienen referencia")
display(df_filtered)

Mostrando los primeros 10 renglones del dataframe


,Organism,AssemblyName,AssemblyStatus,AssemblyAccession,RefSeq_category,FtpPath_GenBank
0,Phaseolus vulgaris (common bean),CIAT_INB841_v1.0,Chromosome,GCA_051991295.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/051...
1,Phaseolus vulgaris (common bean),ASM5162235v1,Complete Genome,GCA_051622355.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/051...
2,Phaseolus vulgaris (common bean),ASM5104807v1,Chromosome,GCA_051048075.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/051...
3,Phaseolus vulgaris (common bean),G12873,Contig,GCA_040616375.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/040...
4,Phaseolus vulgaris (common bean),BAT93,Contig,GCA_040616515.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/040...
5,Phaseolus vulgaris (common bean),JaloEPP558,Contig,GCA_040616365.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/040...
6,Phaseolus vulgaris (common bean),MIDAS,Contig,GCA_040616355.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/040...
7,Phaseolus vulgaris (common bean),P. vulgaris v2.0,Chromosome,GCF_000499845.2,reference genome,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
8,Phaseolus vulgaris (common bean),ASM2944876v1,Chromosome,GCA_029448765.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/029...
9,Phaseolus vulgaris (common bean),ASM1650975v1,Scaffold,GCA_016509755.1,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/016...


Mostrando los genomas que tienen referencia


,Organism,AssemblyName,AssemblyStatus,AssemblyAccession,RefSeq_category,FtpPath_GenBank
7,Phaseolus vulgaris (common bean),P. vulgaris v2.0,Chromosome,GCF_000499845.2,reference genome,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...


In [19]:
genoma_seleccionado = df_filtered.iloc[0]

print("\n--- DETALLES DEL GENOMA SELECCIONADO ---")
print("Organismo:", genoma_seleccionado["Organism"])
print("Nombre del genoma:", genoma_seleccionado["AssemblyName"])
print("Estado del genoma:", genoma_seleccionado["AssemblyStatus"])
print("Accession del genoma:", genoma_seleccionado["AssemblyAccession"])
print("URL de descarga:", genoma_seleccionado["FtpPath_GenBank"])

# cargando las bibliotecas
from ftplib import FTP
from urllib.parse import urlparse

# Seleccionando el path de ftp del genoma de referencia
ftp_url = genoma_seleccionado["FtpPath_GenBank"]

# Obteniendo la información del url
parsed = urlparse(ftp_url)

# Asignando el hostname y el path
ftp_server = parsed.hostname        # 'ftp.ncbi.nlm.nih.gov'
ftp_path = parsed.path

print("Servidor FTP:", ftp_server)
print("Ruta en el servidor:", ftp_path)

# Haciendo la conexión al servidor de forma anónima y moviendose al path del genoma
ftp = FTP(ftp_server)
ftp.login()  # conexión anónima
ftp.cwd(ftp_path)

# Obteniendo la lista de archivos contenidos en el path
files = ftp.nlst()  # lista de archivos en el directorio

# Cerrando la conexión ftp
ftp.quit()

# Imprimiendo la lista de archivos
print("Archivos disponibles:")
for f in files:
    print(f)


--- DETALLES DEL GENOMA SELECCIONADO ---
Organismo: Phaseolus vulgaris (common bean)
Nombre del genoma: P. vulgaris v2.0
Estado del genoma: Chromosome
Accession del genoma: GCF_000499845.2
URL de descarga: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/499/845/GCA_000499845.2_P._vulgaris_v2.0
Servidor FTP: ftp.ncbi.nlm.nih.gov
Ruta en el servidor: /genomes/all/GCA/000/499/845/GCA_000499845.2_P._vulgaris_v2.0


Archivos disponibles:
annotation_hashes.txt
GCA_000499845.2_P._vulgaris_v2.0_assembly_structure
README.txt
GCA_000499845.2_P._vulgaris_v2.0_assembly_report.txt
GCA_000499845.2_P._vulgaris_v2.0_assembly_stats.txt
GCA_000499845.2_P._vulgaris_v2.0_cds_from_genomic.fna.gz
GCA_000499845.2_P._vulgaris_v2.0_fcs_report.txt
GCA_000499845.2_P._vulgaris_v2.0_feature_count.txt
GCA_000499845.2_P._vulgaris_v2.0_feature_table.txt.gz
GCA_000499845.2_P._vulgaris_v2.0_genomic.fna.gz
GCA_000499845.2_P._vulgaris_v2.0_genomic.gbff.gz
GCA_000499845.2_P._vulgaris_v2.0_genomic.gff.gz
GCA_000499845.2_P._vulgaris_v2.0_genomic.gtf.gz
GCA_000499845.2_P._vulgaris_v2.0_genomic_gaps.txt.gz
GCA_000499845.2_P._vulgaris_v2.0_protein.faa.gz
GCA_000499845.2_P._vulgaris_v2.0_protein.gpff.gz
GCA_000499845.2_P._vulgaris_v2.0_rna_from_genomic.fna.gz
GCA_000499845.2_P._vulgaris_v2.0_translated_cds.faa.gz
GCA_000499845.2_P._vulgaris_v2.0_wgsmaster.gbff.gz
md5checksums.txt
assembly_status.txt
uncompressed_checksums.txt


In [20]:
# Obtener archivos FASTA y GFF
fasta_files = [f for f in files if f.endswith("_genomic.fna.gz")]
gff_files = [f for f in files if f.endswith("_genomic.gff.gz")]

print("Archivos FASTA disponibles:", fasta_files)
print("Archivos GFF disponibles:", gff_files)


Archivos FASTA disponibles: ['GCA_000499845.2_P._vulgaris_v2.0_cds_from_genomic.fna.gz', 'GCA_000499845.2_P._vulgaris_v2.0_genomic.fna.gz', 'GCA_000499845.2_P._vulgaris_v2.0_rna_from_genomic.fna.gz']
Archivos GFF disponibles: ['GCA_000499845.2_P._vulgaris_v2.0_genomic.gff.gz']


In [21]:
import urllib.request
import os

if fasta_files or gff_files:
    # Carpeta donde guardaremos
    out_dir = "/export/space3/users/darapc/biopythonCLASS/RbohB-DE/data/reference"
    os.makedirs(out_dir, exist_ok=True)
    # Descarga de archivo FASTA
    if fasta_files:
        # Obtener el nombre base del ensamblaje (ej. GCA_000001215.4_Drosophila_melanogaster...)
        fasta_url = ftp_url + "/" + fasta_files[0]
        print("FASTA:", fasta_url)
        # Archivo de salida
        fasta_out = os.path.join(out_dir, fasta_files[0])
        # Descargar FASTA
        urllib.request.urlretrieve(fasta_url, fasta_out)
        print("Genoma descargado en:", fasta_out)
    # Descarga de archivo GFF    
    if gff_files:
        gff_url = ftp_url + "/" + gff_files[0]
        print("GFF:", gff_url)
        # Archivo de salida
        gff_out = os.path.join(out_dir,gff_files[0])
        # 4. Descargamos el GFF
        urllib.request.urlretrieve(gff_url, gff_out)
        print("Archivo GFF descargado en:", gff_out)

FASTA: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/499/845/GCA_000499845.2_P._vulgaris_v2.0/GCA_000499845.2_P._vulgaris_v2.0_cds_from_genomic.fna.gz
Genoma descargado en: /export/space3/users/darapc/biopythonCLASS/RbohB-DE/data/reference/GCA_000499845.2_P._vulgaris_v2.0_cds_from_genomic.fna.gz
GFF: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/499/845/GCA_000499845.2_P._vulgaris_v2.0/GCA_000499845.2_P._vulgaris_v2.0_genomic.gff.gz
Archivo GFF descargado en: /export/space3/users/darapc/biopythonCLASS/RbohB-DE/data/reference/GCA_000499845.2_P._vulgaris_v2.0_genomic.gff.gz
